<a href="https://colab.research.google.com/github/MU-AdelAsaad/Toon09-Crew17_spaceMissionX01/blob/main/Copy_of_lab2_empty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP 2024
# Lab 2: Word Embeddings

Throughout the course we have discussed different ways to represent a word. The latest one (and the most successful in terms of results) is using word embeddings: we represent each word as a vector consisting of numbers. The vector encodes the meaning of the word. These numbers (or weights) for each word are learned using various machine learning models. In this lab, we explore how to create such vectors given a corpus (note that in the real world, you can also load the trained word vectors, and you will almost never have to train them from scratch), which properties they have and what kind of tasks we can solve with them.

By the end of this lab you should be able to:
- Implement and/or use built-in functions to preprocess your data (once again)
- Execute `word2vec` based on a corpus
- Inspect and test word embeddings properties
- Use word embeddings to get sentence representations (aka sentence embeddings)
- Use sentence embeddings to solve more complicated tasks like information retrieval
- Design evaluation frameworks for specific NLP tasks and assess their difficulty

### Score breakdown

Exercise | Points
--- | ---
[Exercise 1](#e1) | 3
[Exercise 2](#e2) | 3
[Exercise 3](#e3) | 3
[Exercise 4](#e4) | 5
[Exercise 5](#e5) | 10
[Exercise 6](#e6) | 6
[Exercise 7](#e7) | 5
[Exercise 8](#e8) | 5
[Exercise 9](#e9) | 5
[Exercise 10](#e10) | 5
[Exercise 11](#e11) | 5
[Exercise 12](#e12) | 15
Total | 70

This score will be scaled down to 1 and that will be your final lab score.

### Instructions for delivery (Deadline: 17/May late night, wildcards possible)

+ Make sure that you include a proper amount/mix of comments, results and code.
+ In the end, make sure that all cells are executed properly and everything you need to show is in your (execucted) notebook.
+ You are asked to deliver only your executed notebook file, .ipnyb and nothing else. Enjoy!
+ Honor code applies to these tasks. Only individual work should be submitted.
+ While you may talk with others about this lab, we ask that you write your solutions individually. If you do discuss specific tasks with others please include their names below.
+ It is mandatory to list and disclose any website (or other resource) you used (e.g. stackoverflow) as well as any genAI tools (e.g. chatGPT) used.

Collaborators: list collaborators here

**I talked with Jerry about...**

Use of genAI tools (e.g. chatGPT), websites (e.g. stackoverflow): list websites where you found code (or other info) as well as include information on how you used genAI tools (e.g. prompts):

I asked chatGPT about...

## 0. Setup

As in the last lab, we will be using huggingface datasets library ([https://huggingface.co/datasets](https://huggingface.co/datasets)). You can find the detailed documentation and tutorials here: [https://huggingface.co/docs/datasets/en/index](https://huggingface.co/docs/datasets/en/index)

If you don't have it installed you can run the code below:

In [ ]:
! pip install -U datasets~=2.18.0

As usual, we start by importing some essential Python libraries and we will be using. Apart from `gensim` (which is going to be used for word2vec), we have already seen the others.

In [ ]:
import re  # for regular expressions
import random
import math

import numpy as np
import matplotlib.pyplot as plt
import datasets
import gensim
import nltk
import tqdm

nltk.download('punkt')

## 1. Load and Preprocess Data

*Sentence compression* involves rephrasing sentences to make them shorter while still retaining the original meaning. A reliable compression system would be valuable for mobile devices and could also serve as a component in an extractive summarization system.

The dataset we are going to use can be found on [Huggingface](https://huggingface.co/datasets/embedding-data/sentence-compression). It concerns a set of 180,000 pairs of sentences, aka it is a parallel corpus of sentences and their equivalent compressions. It has been collected by harvesting news articles from the Internet where the headline appears to be similar to the first sentence and that property is used to find an "extractive" compression of the sentence.

For example, for the sentence

`"Regulators Friday shut down a small Florida bank, bringing to 119 the number of US bank failures this year amid mounting loan defaults"`

the compressed equivalent (based on the dataset) is:

`"Regulators shut down small Florida bank"`.


For more information you can read the original paper (from Google) [here](https://aclanthology.org/D13-1155.pdf). We strongly recommend going over the paper to gain further insights. Notice that the paper is from 2013, therefore word embeddings have not been widely introduced yet in NLP tasks, meaning that the methods applied were based on the traditional NLP pipeline (feature extraction + ML).

### 1.1 Loading the Dataset

The dataset will be loaded as a Pandas DataFrame. This may take a few minutes because of the large size of the data.

Make sure to inspect the dataset and make sure it is imported properly.

In [ ]:
ds = datasets.load_dataset('embedding-data/sentence-compression')
print(ds)

In [ ]:
for i in range(10):
    print(ds['train'][i])

The dataset comes with only the `train` split so we will have to split it ourselves.

In [ ]:
split_ds = ds['train'].train_test_split(test_size=0.2)
print(split_ds)

### 1.2 Preprocessing the dataset
In this section we will prepare the dataset, aka clean the sentences and tokenizem.

First, let's write the function to clean the text. It can be similar to the one from the previous lab (Lab1) but make sure that it makes sense for this dataset and task.

More specifically, think about lower-casing, punctuation, stop-words and lemmatization/stemming and the impact it might have on the dataset. Also reflect on the fact that with word2vec we want to uncover semantic relationships between words, whereas with bag-of-words we were trying to capture different morphological variations.

<a name='e1'></a>
### Exercise 1 (points 3)
Fill in the following function ot clean the dataset.

In [ ]:
def clean(text):
    """
    Cleans the given text
    Args:
        text: a str with the text to clean

    Returns: a str with the cleaned text

    """

    # Empty text
    if type(text) not in (str, np.str_) or text == '':
        print('empty text', type(text), type(text) not in (str, np.str_), text)
        return ''

    # 'text' from the example can be of type numpy.str_, let's convert it to a python str
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub("\'s", " ",
                  text)  # we have cases like "Sam is" or "Sam's" (i.e. his) these two cases aren't separable, I choose to compromise are kill "'s" directly
    text = re.sub(" whats ", " what is ", text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ", text)

    #you might need more
    #add them here

    ### YOUR CODE HERE




    ### YOUR CODE ENDS HERE

    # remove comma between numbers, i.e. 15,000 -> 15000
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)

    text = text.strip()

    # Update the example with the cleaned text
    return text

The following function will apply the function (sic) you just wrote to the whole dataset. More specifically, it takes the first entry (`sentence`) from the set of uncompressed/compressed pairs, applies the `clean` function and saves the processed sentence in the field `clean_sentence`. The same is dome for the compressed version of the sentence (saved as `clean_compressed`).

In [ ]:
def clean_dataset(example):
    """
    Cleans the sentence and compressed sentence in the example from the Dataset
    Args:
        example: an example from the Dataset

    Returns: updated example with 'clean_sentence' and 'clean_compressed' cleaned

    """
    sentence, compressed = example['set']
    clean_sentence = clean(sentence)
    clean_compressed = clean(compressed)
    example['clean_sentence'] = clean_sentence
    example['clean_compressed'] = clean_compressed
    return example

Below we apply the function to the whole dataset (using `map`) and we can also inspect the result.

In [ ]:
split_ds = split_ds.map(clean_dataset)
print(split_ds)

Let's examine some examples from the dataset and make sure that we got the results we wanted. At this step, it might be necessary to revisit some pre-processing steps if you are not happy with the results.

In [ ]:
for i in range(10):
    print(split_ds['train'][i])

<a name='e2'></a>
### Exercise 2 (points 3)

As always, we will need to tokenize the dataset in order to train our own `word2Vec` model in the next section. We will use the [Natural Language Toolkit (NLTK) library]([https://www.nltk.org/]) (https://www.nltk.org/).

Complete the following function to split the text into tokens using the `word_tokenize()` function. Check the [documentation](https://www.nltk.org/api/nltk.tokenize.word_tokenize.html?highlight=word_tokenize) first.

Note that there are different tokenizers e.g. `RegexpTokenizer` where you can enter your own regexp, `WhitespaceTokenizer` (similar to Python's string.split()) and `BlanklineTokenizer`.

In [ ]:
# if you want you can remove the stopwords
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))

def tokenize(text):
    """
    Tokenizes the `text` parameter using nltk library
    Args:
        text: a string representing a sentence to be tokenized

    Returns: a list of tokens (strings)

    """

    ### YOUR CODE HERE





    ### YOUR CODE ENDS HERE
    return tokens

Next, the function will be applied to the whole dataset (as we did with the pre-processing) and `sentence_tokens` field will be created to store the result.

In [ ]:
def tokenize_dataset(example):
    """
    Tokenizes 'clean_sentence' columns in the example from the Dataset
    Args:
        example: an example from the Dataset

    Returns: updated example with 'sentence_tokens' columns

    """
    cleaned_sentence = example['clean_sentence']
    tokens = tokenize(cleaned_sentence)
    example['sentence_tokens'] = tokens
    return example

In [ ]:
split_ds = split_ds.map(tokenize_dataset)

In [ ]:
for i in range(10):
    print(split_ds['train'][i])

Since we will need the tokenized sentences, we can use the following statement to extract them from the `train` split of our dataset.

In [ ]:
tokenized_sentences = split_ds['train']['sentence_tokens']
print(len(tokenized_sentences))
print(tokenized_sentences[:10])

Notice the difference in the types of the different structures we use. Run the following cell to check the types. Do they make sense to you?

In [ ]:
#type of original dataset
print(type(split_ds))
print("--")
#type of original sentence
print(split_ds['train'][1])
print(type(split_ds['train'][1]))
print("--")
#type of pre-proceesed sentence
print(split_ds['train']['clean_sentence'][1])
print(type(split_ds['train']['clean_sentence'][1]))
print("--")
#type of tokenized sentence
print(split_ds['train']['sentence_tokens'][1])
print(type(split_ds['train']['sentence_tokens'][1]))
print("--")

## 2. Word2Vec word embeddings

Before proceeding below make sure that you understand the way Word2Vec words. Refer to the lecture slides and/or the textbooks. Here we provide a (very) rough introduction that is lacking many details we discussed in class.

Word2vec operates on the principle that words appearing in similar contexts tend to have similar meanings. There are two main algoriths with diffferent variations. in Skip-gram the model precits the context words given the target word, whereas in CBOW the model predicts the target word given its context words (words surrounding it). During training (which can take a lot of time in case of big corpora), word2Vec updates the vector representations of words to maximize the probability of correctly predicting the target or context words. If we use a neural network, training is done with gradient descent.

We need to decide on different parameters that will influence the behavior and performance of word2Vec models. These include the dimensionality of the word vectors which determines the size of the vector space in which words are embedded, and the window (context) size, which defines the number of context words considered in each training instance. Additionally, the choice of training algorithm (CBOW or Skip-gram), learning rate, number of training epochs need to be tuned.

Nough said, it's time to train our own Word2Vec embeddings. We will use the powerful [Gensim library](https://radimrehurek.com/gensim/models/word2vec.html). Make sure to read the documentation to decrypt the parameters we need to tune (embedding vector size, context window, type of algorithm etc.). The training might take some time, depending on the parameters you use.



In [ ]:
from gensim.models import Word2Vec

embedding_size = 100
w2v_model = Word2Vec(tokenized_sentences, vector_size=embedding_size, min_count=5, window=5, sg=1, hs=0, seed=1,
                     workers=4)

In [ ]:
w2v_model.train(tokenized_sentences, total_examples=len(tokenized_sentences), epochs=10)

The Word2Vec model has `wv` attribute. We can use it to retrieve the whole vocabulary (aka for how many words we learned embeddings for).

In [ ]:
vocab = list(w2v_model.wv.key_to_index)
print(len(vocab))

Here you can see the word cloud created from the vocabulary.

In [ ]:
# visualizing our vocab with a word cloud.
from wordcloud import WordCloud

# Generate a word cloud image for words
combined_vocab = ' '.join(vocab)
wordcloud = WordCloud(width=600, height=400).generate(combined_vocab)
plt.figure(figsize=(9, 9))
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout()
plt.show()

Let's explore a bit further the embeddings. In the following cells, the embedding of a single word is returned. Double-check the dimensions (as sanity check). This is like inspecting the `W` matrix (weights) that we discussed in the lecture.

In [ ]:
# vector of a particular model. note that it is 100 dimensional as specified.
w2v_model.wv['what']

In [ ]:
# can also do like this.
w2v_model.wv['apple']

`word2vec` offers different functions to easily run very common tasks. For example, there are different functions to find the most similar words.

Check the documentation on how [`most_similar`](https://tedboy.github.io/nlps/generated/generated/gensim.models.Word2Vec.most_similar.html) and [`similar_by_word`](https://tedboy.github.io/nlps/generated/generated/gensim.models.Word2Vec.similar_by_word.html) can be used.

In [ ]:
# most similar words to a given word
print(w2v_model.wv.most_similar('what', topn=10))

# also u can use
print(w2v_model.wv.similar_by_word('miss', topn=5))

In [ ]:
print(w2v_model.wv.most_similar('why', topn=10))

In [ ]:
print(w2v_model.wv.similar_by_word('who', topn=5))

<a name='e3'></a>
### Exercise 3 (points 3)

While there is already a similarity metric in the `word2Vec` class, we will write our own implementation of the cosine similarity. Complete the following funciton that given any two vectors will compute the cosine similarity. If you don't remember the formula for the cosine similarity, revisit the course material. Notice that the function receives numpy arrays and recall that you can express cosine similarity as a dot product. Use numpy functions to write an efficient implementation.

In [ ]:
def cosine_similarity(vector1, vector2):
    """
    Computes the cosine similarity between two vectors
    Args:
        vector1: numpy array of the first vector
        vector2: numpy array of the second vector

    Returns: cosine similarity

    """
    ### YOUR CODE HERE





    ### YOUR CODE ENDS HERE

In [ ]:
cosine_similarity(np.array([0, 1, 2]), np.array([0, 1, 2]))

We can now compare our implementation with the one in the Word2Vec model and confirm what we already expected.

In [ ]:
# simalarity between two words
word1 = 'alive'
word2 = 'biology'
print(w2v_model.wv.similarity(word1, word2))
print(cosine_similarity(w2v_model.wv[word1], w2v_model.wv[word2]))

In [ ]:
# simalarity between two words. similar words
word1 = 'alive'
word2 = 'life'
print(w2v_model.wv.similarity(word1, word2))
print(cosine_similarity(w2v_model.wv[word1], w2v_model.wv[word2]))

In [ ]:
# simalarity between two words. dissimilar words
word1 = 'alive'
word2 = 'dead'
print(w2v_model.wv.similarity(word1, word2))
print(cosine_similarity(w2v_model.wv[word1], w2v_model.wv[word2]))

In [ ]:
# simalarity between two words. unrelated words
word1 = 'alive'
word2 = 'horse'
print(w2v_model.wv.similarity(word1, word2))
print(cosine_similarity(w2v_model.wv[word1], w2v_model.wv[word2]))

In [ ]:
# simalarity between two SAME words
w2v_model.wv.similarity('equal', 'equal')
word1 = 'equal'
word2 = 'equal'
print(w2v_model.wv.similarity(word1, word2))
print(cosine_similarity(w2v_model.wv[word1], w2v_model.wv[word2]))

The next function contains the code to plot a similarity matrix between multiple words (e.g. if we want to compare 10 words and their pair-wise similarities). It requires a matrix with similarities (as input) and labels (aka the words) to display in the final figure.

In [ ]:
def plot_similarity_matrix(matrix, labels):
    """
    Displays a plot of the `matrix` of size (N x N) with the labels specified as a list of size N
    Args:
        matrix: a square-sized (N x N) numpy array
        labels: a list of strings of hte size N
    """

    fig, ax = plt.subplots()
    im = ax.imshow(matrix)

    # Show all ticks and label them with the respective list entries
    ax.set_xticks(np.arange(len(labels)), labels=labels)
    ax.set_yticks(np.arange(len(labels)), labels=labels)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    for i in range(len(labels)):
        for j in range(len(labels)):
            text = ax.text(j, i, f'{matrix[i, j]:.2f}',
                           ha="center", va="center", color="w")

    # ax.set_title("Give a title if you want")
    fig.tight_layout()
    plt.show()

<a name='e4'></a>
### Exercise 4 (points 5)

In the following, we will explore some properties of word embeddings through some examples. We will use 6 example words for this purpose but feel free to explore some of your own (also for Exercise 5).

First, fill in the next cell to create a similarity matrix between a list of words. Then, call the function given above that plots the similarity matrix, along with the word labels.

In the cell, below comment on the results. Based on the word2vec parameters you explored, do they make sense?

In [ ]:
list_of_words = ['love', 'hate', 'life', 'equal', 'alive', 'dead']

similarity_matrix = np.zeros((len(list_of_words), len(list_of_words)), dtype=float)

### YOUR CODE HERE




### YOUR CODE ENDS HERE


plot_similarity_matrix(similarity_matrix, list_of_words)

<a name='e5'></a>
### Exercise 5 (points 10)

For this exercise, experiment with different words and their similarities plotted. Also, take into account the different parameters of word2vec and more specifically the window size (context) and the role it plays in the types of embeddings we can learn.

What happens if I pick a small context window?
What happens if I pick a large context window?

Are there noticeable patterns? Are they expected? Try to find examples that word embeddings make sense (based on your hypothesis) and not.

Comment on your findings below.

In [ ]:
#### YOUR CODE HERE




### YOUR CODE ENDS HERE

// your comments

## 3. Pre-trained embeddings

In this short section you will load the pre-trained embeddings of your choice. Gensim library maintains a storage containing some pre-trained models. You can read more about it [here](https://github.com/piskvorky/gensim-data) ([https://github.com/piskvorky/gensim-data](https://github.com/piskvorky/gensim-data)). Be sure to read the README of this repository.

Let's first load the info of what models are available.

In [ ]:
import json
import gensim.downloader as api

info = api.info()  # show info about available models/datasets
print(json.dumps(info['models'], indent=2))

<a name='e6'></a>
### Exercise 6 (points 6)

Load the pre-trained word embeddings model of your choice (like we did in the lecture: e.g. try different pre-trained corpora and different types of embeddings (like GloVe or FastText)). You can use the gensim implementation and functionality to download and load several types of embeddings, but you are not limited to it.

Next, perform the analysis of the similarities between words as you did in Exercise 5. Compare your trained word2vec with the pretrained model you downloaded. Try to explain the numerical differences based on what you know on how these models were trained.

Note: Pre-trained embeddings might be difficult to load and use (memory requirements) so you might have to work locally there. You don't need to submit (really, don't!) the pre-trained word embeddings you used but make sure to include a link in your delivered notebook.

In [ ]:
### YOUR CODE HERE




### YOUR CODE ENDS HERE

// your comments

If you encounter problems with not enough RAM you can modify and run the following cell to unload the pretrained model from memory. Ofcourse, change the variable name to whatever you used.

In [ ]:
# del fasttext_model

## 4. Sentence Embeddings by Averaging Word2Vec Embeddings

Word embeddings are a powerful model for representing words and their meaning (in terms of distributional similarity). As we discussed in class, we can use them in a wide variety of tasks with more complex architectures. Word vectors offer a dense vector for each word. What if we wanted to represent a sentence (or a document) based on word vectors. How can we do that?

In the course, we will see different architectures that take into account the sequence of words (by combining their vectors). A first naive but simple and sometimes (as we are going to see) quite effective approach would be to represent a sentence with an embedding vector that is the average of the word vectors that form the sentence.

So formally, this is what we are aiming for:

$
\text{Sentence_Embedding} = \frac{1}{N} \sum_{i=1}^{N} \text{Word_Embedding}_i
$

where:
* $N$ is the number of words in a sentence
* $\text{Word_Embedding}_i$ is the word vector for the $i$-th in the sentence.

Things to note:
* The embedding vector for the sentence will obviously have the same dimension as the word embedding.
* This representation ignores the word order (like bag-of-words). During the course we will see how we can overcome this limitation by using sequence models.

<a name='e7'></a>
### Exercise 7 (points 5)

Complete the function below that takes as input the sentence in the form of tokens (so it's a list of words) and calculates the sentence embedding vector. First, we would need to retrieve the word embeddings for each word from our trained Word2Vec model and then average the vectors.

Note: There can be cases where all tokens from a sentence are out-of-vocabulary words (OOV). Return None in that case. We will filter out the sentences where it occurs.

In [ ]:
def embed_sentence_word2vec(tokens):
    """
    Calculates the sentence embedding by averaging the embeddings of the tokens
    Args:
        tokens: a list of words from the sentence

    Returns: a numpy array of the sentence embedding

    """
    #### YOUR CODE HERE
    #### CAUTION: be sure to cover the case where all tokens are out-of-vocabulary!!!
    #### Return None in that case - we will filter out those cases later.





    ### YOUR CODE ENDS HERE

Now we can apply the function to the whole dataset. Here we do it both for the sentence and the compressed version. You should know it by now, but this operation might take some time. The next cells will apply your function to the whole dataset.

In [ ]:
def embed_sentence_word2vec_dataset(example):
    """
    Embeds the sentence and the compressed sentence in the example from the Dataset
    Args:
        example: an example from the Dataset

    Returns: updated example with 'sentence_word2vec' and 'compressed_word2vec' columns

    """
    sentence_tokens = example['sentence_tokens']
    clean_compressed = example['clean_compressed']
    compressed_tokens = tokenize(clean_compressed)

    sentence_embedding = embed_sentence_word2vec(sentence_tokens)
    compressed_embedding = embed_sentence_word2vec(compressed_tokens)

    example['sentence_word2vec'] = sentence_embedding
    example['compressed_word2vec'] = compressed_embedding
    return example

In [ ]:
split_ds = split_ds.map(embed_sentence_word2vec_dataset)
print(split_ds)

Here we will filter out the examples where sentences (or their compressed versions) are composed of only OOV words. Notice the changed size of the dataset after the operation is complete.

In [ ]:
split_ds = split_ds.filter(lambda e: e['sentence_word2vec'] is not None and e['compressed_word2vec'] is not None)
print(split_ds)

In [ ]:
for i in range(10):
    print(split_ds['train'][i])

Huggingface datasets can return the columns as different types. Our embeddings are numpy arrays and it is a lot faster to return them as their type. The next cell will cast our dataset to return columns as numpy arrays.

In [ ]:
split_ds.reset_format()
np_ds = split_ds.with_format('np', columns=['sentence_word2vec', 'compressed_word2vec'], dtype=float)


Here you can see that the new dataset returned a single numpy array containing all sentence embeddings in our dataset. This is a lot more efficient than returning a list of arrays (which is the default behaviour). Below we check the type and the dimensionality.

We will be using `text` subset from our dataset to not use too much RAM. But if your system has enough memory, try to use `train` subset here.

In [ ]:
sent_word2vec = np_ds['test']['sentence_word2vec']
compr_word2vec = np_ds['test']['compressed_word2vec']
print(type(sent_word2vec))
print(sent_word2vec.shape)
print(type(compr_word2vec))
print(compr_word2vec.shape)

## 5. Retrieving Sentences
In this section we will try use the vector representations in our dataset to retrieve only the relevant ones based on some user query. You can think of it as a search retrieval task (based on what we discussed in the relevant lecture) where a user provides a query (that is the compressed sentence) and the system returns the sentences that are more similar to the query.

In the information retrieval lecture, we discussed how to solve this retrieval problem by using bag-of-words as a representation basis. Here, we will use word embeddings instead.

<a name='e8'></a>
### Exercise 8 (points 5)

First step to a retrieval task is to embed the query (aka find a representation). We will do it the same way as we did for the sentences in our dataset. Complete the following function to return the embedding of the provided text.

In [ ]:
def embed_query_word2vec(query):
    """
    Embeds the provided query using trained Word2Vec model.
    The query is first cleaned and tokenized (with the `clean()` and `tokenize() functions.
    Args:
        query: a str with the query

    Returns: a numpy array with the embedded qury

    """

    #### YOUR CODE HERE





    ### YOUR CODE ENDS HERE

Next we try the condensed representatin based on a simple query. Feel free to try different queries with different words. What happens if we have OOV words in a query?

In [ ]:
query = "volcano erupted"
print(query)

query_word2vec = embed_query_word2vec(query)
print(query_word2vec.shape)
print(query_word2vec)

<a name='e9'></a>
### Exercise 9 (points 5)

The next step in our retrieval system, would be to calculate the proximity of a query to our retrieval corpus (in our case that is all the sentences).

Complete the following function to calculate the cosine similarity between a vector (first parameter `vector`, that will usually be the query vector) and all other vectors (second parameter `other_vectors`, that will be the sentence embeddings in our case). Note that the `other_vectors` parameter is a single numpy array of size `N x D`, where $N$ is the number of vectors and $D$ is the dimension of each vector.

For maximum efficiency (we will need it) do not use loops. Try to write the implementation with numpy functions. Hint: matrix multiplication can be seen as calculating the dot product between rows and columns of the multiplied matrices.

In [ ]:
def cosine_similarity_1_to_n(vector, other_vectors):
    """
    Calculates the cosine similarity between a single vector and other vectors.
    Args:
        vector: a numpy array representing a vector of D dimensions
        other_vectors: a 2D numpy array representing other vectors (of the size NxD, where N is the number of vectors and D is their dimension)

    Returns: a 1D numpy array of size N containing the cosine similarity between the vector and all the other vectors

    """

    #### YOUR CODE HERE






    ### YOUR CODE ENDS HERE

We will use the function to calculate the similarity of all sentences in the dataset to our query.

In [ ]:
query_similarity = cosine_similarity_1_to_n(query_word2vec, sent_word2vec)
print(query_similarity.shape)
print(query_similarity[:10])

The following cell will select the most similar sentence.

In [ ]:
most_similar = int(np.argmax(query_similarity))
print(most_similar)
print(query_similarity[most_similar])
print(split_ds['test'][most_similar]['set'][0])

The following function will return the indices of the top-k elements in the array.

In [ ]:
def top_k_indices(array, k, sorted=True):
    """
    Returns top-k indices from the 1D array. If `sorted` is `True` the returned indices are sorted in the descending order
    Args:
        array: a 1D numpy array
        k: a number of top indices to return
        sorted: if True, the returned indices are sorted in descending order

    Returns: a 1D array containing top-k indices

    """
    top_k = np.argpartition(array, -k)[-k:]
    if sorted:
      selected = array[top_k]
      sorted_selected = (-selected).argsort()
      top_k = top_k[sorted_selected]
    return top_k

In [ ]:
top_indices = top_k_indices(query_similarity, k=10).tolist()
for idx in top_indices:
    print(split_ds['test'][idx]['set'][0])
    print(f'similarity: {query_similarity[idx]}')

<a name='e10'></a>
### Exercise 10 (points 5)

Experiment with different queries (taking into account the nature of the dataset and your insights from the analysis so far).

Does the search perform well? When does it fail? Discuss several examples that are we get an expected but also unexpected results (find at least 5 from each category). Try to provide reasons for the good/bad result in each case (e.g. is there some error in the data, is there some linguistic phenomenon that we don't capture, is something wrong with our modeling with average embeddings, ...)

In [ ]:
#### YOUR CODE HERE




### YOUR CODE ENDS HERE

// your comments

## 5. Evaluating Retrieval

In this last section we will try to evaluate how good our sentence retrieval system is. To keep the computational resources manageable, we will use the test set for that as its size is more manageable.

Recall from the lecture in IR that there are several metrics to evaluate retrieval performance by taking into account the relevance of the retrieved results to the query. We will use Recall@K here (for more metrics and more details refer to the lecture slides and the textbooks).

RRecall@K is a metric used to measure the effectiveness of a search system in retrieving relevant documents within the top $K$ retrieved documents. It calculates the proportion of relevant documents retrieved within the top-$K$ results, compared to the total number of relevant documents in the collection.

$
\text{Recall@K} = \frac{\text{Number of relevant documents retrieved in the top }-K}{\text{Total number of relevant documents}}
$

In our case, we have a sentence, and it's compressed version. To test our system, we will treat compressed sentences as the queries. Each query will have only a single relevant sentence - the corresponding uncompressed sentence.

Therefore, for the calculation of Recall@K we will take into account whether the correct retrieved result is contained within the first $K$ retrieved results. For example, if for a query (i.e. a compressed sentence) we retrieve 10 results and within these we see the relevant one (i.e. the full sentence), then Recall@10 = 1.

<a name='e11'></a>
### Exercise 11 (points 5)

In this exercise you will revisit your implementation of the cosine siliarity. Generalize it so that it can accept two arrays containing two sets of vectors (first one containing $M$ vectors and the second one $N$ vectors). Compute the cosine similarity between each pair of vectors coming from the two sets. The result should be an array of size $M x N$.

Once again, try to write an efficient code. This means no loops. Remember the relation between matrix multiplication and dot product. (Depending on your implementation of the previous function calculating cosine similarity, this one can be almost the same)

In [ ]:
def cosine_similarity_m_to_n(vectors, other_vectors):
    """
    Calculates the cosine similarity between a multiple vectors and other vectors.
    Args:
        vectors: a numpy array representing M number of vectors of D dimensions (of the size MxD)
        other_vectors: a 2D numpy array representing other vectors (of the size NxD, where N is the number of vectors and D is their dimension)

    Returns: a numpy array of cosine similarity between all the vectors and all the other vectors

    """

    #### YOUR CODE HERE





    ### YOUR CODE ENDS HERE

The following function will use your implementation to calculate Recall@K based on the similarity matrix.

In [ ]:
def calculate_recall(queries, sentences, k, batch_size=1000):
  """
  Calculates recall@k given the embeddings of the queries and sentences.
  Assumes that only a single sentence with the same index as query is relevant.
  Batching is implemented to avoid high memory usage.
  Args:
      queries: a numpy array with the embeddings of N queries
      sentences: a numpy array with the embeddings of N sentences available for retrieval
      k: number of top results to search for the relevant sentence
      batch_size: number of queries to process at a time

  Returns: calculated recall@k

  """
  n_queries = queries.shape[0]
  if batch_size is None:
    batch_size = n_queries

  n_batches = math.ceil(n_queries / batch_size)
  last_batch_size = n_queries % batch_size if n_queries != batch_size else batch_size

  correct = np.zeros(n_queries).astype(bool)
  with tqdm.tqdm(total=n_queries) as pbar:
    for b in range(n_batches):
      effective_batch_size = last_batch_size if b == (n_batches - 1) else batch_size
      batch_start_index = b * batch_size

      queries_batch = queries[batch_start_index:batch_start_index + effective_batch_size]
      batch_similarity = cosine_similarity_m_to_n(queries_batch, sentences)

      for i in range(effective_batch_size):
        query_index = i + batch_start_index
        query_similarity = batch_similarity[i]
        top_k = top_k_indices(query_similarity, k=k, sorted=False)

        if query_index in top_k:
            correct[query_index] = True

        pbar.update(1)

  n_correct = np.sum(correct)
  n_total = correct.shape[0]
  recall = n_correct / n_total
  return recall

You can use it like so:

In [ ]:
recall_at_1 = calculate_recall(compr_word2vec, sent_word2vec, k=1, batch_size=1000)
print(f'\n{recall_at_1 * 100:.2f}%' )

<a name='e12'></a>
### Exercise 12 (points 5+10)

(5 points) Calculate recall for different values of $K$. Comment on how recall changes based on the value of $K$. Are the results expected or surprising? Comment again on different examples (like in Exercise 10) but now take into account the results of recall at different levels of $K$.

Open question (10 points): Try to improve the scores by e.g. tuning the word2vec parameters or using pre-trained embeddings. Discuss the results you achieve, even if you didn't manage to improve the scores.

Note: Pre-trained embeddings might be difficult to load and use (memory requirements) so you might have to work locally there. You don't need to submit (really, don't!) the pre-trained word embeddings you used but make sure to include a link in your delivered notebook.

In [ ]:
#### YOUR CODE HERE




### YOUR CODE ENDS HERE

// your comments